In [1]:
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator

Loading Fashion MNIST Dataset

In [ ]:
num_classes = 10
img_rows , img_cols = 28, 28

mnist = tf.keras.datasets.fashion_mnist
# splitting dataset into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Data Preprocessing & CNN Model Architecture

In [ ]:
if k.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'Training Samples')
print(x_test.shape[0], 'Testing Samples')

# converting class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))
model.summary()

x_train shape: (60000, 28, 28, 1)
60000 Training Samples
10000 Testing Samples
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                           

Model Training & Optimization

In [ ]:
opt = Adam(learning_rate=0.0005)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    x_train, y_train, batch_size=32,
    steps_per_epoch=len(x_train) // 32,
    epochs=50,
    verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

Model Evaluation

In [5]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.2179
Test Accuracy: 0.9235


In [7]:
model.save("model_cnn.keras")
print("Model Saved")

Model Saved


Loading Saved Model & Testing Sample Image (Successful)

In [ ]:
from keras.utils import load_img, img_to_array
from keras.models import load_model
import numpy as np

# loading and preparing image
def load_image(filename):
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	img = img_to_array(img)
	img = img.reshape(1, 28, 28, 1)
	img = img.astype('float32')
	img = img / 255.0
	return img

# loading image and predicting the class
def run_example():
	img = load_image('./content/sample_image.png')
	model = load_model('./content/model_cnn.keras')
	result = np.argmax(model.predict(img), axis=-1)
	print(result[0])

run_example()

1/1 [==============================] - 0s 49ms/step
2
